In [1]:
from triqs.gf import *
from triqs.plot.mpl_interface import *
from triqs.lattice import *
from triqs.gf.tools import *
from triqs_tprf.lattice import lattice_dyson_g0_wk
from triqs_tprf.lattice import lattice_dyson_g_wk
# from triqs_tprf.lattice_utils import imtime_bubble_chi0_wk
from triqs_tprf.tight_binding import TBLattice
import numpy as np
from scipy.integrate import quad
import gc
from tqdm import tqdm
from IPython.display import display, clear_output

pi = np.pi
g = 1.0

Starting serial run at: 2023-11-26 21:06:22.633470


We can import the dressed Green's function from file. 
For the vertex, we have the PH vertex from the susceptibility, we need to compute the PP vertex. 
Then we set up the Eliashberg equations and solve them.

In [2]:
from h5 import HDFArchive
with HDFArchive('chi_expt_niw36.h5', 'r') as f:
    chi_expt_kiO = f['chi_expt'] 
with HDFArchive('Gkiw.h5', 'r') as f:
    G_kiw = f['G_kiw']

In [3]:
kiOmegamesh=chi_expt_kiO.mesh
kiwmesh=G_kiw.mesh
niO=kiOmegamesh[1].last_index()
niw=-kiwmesh[1].first_index()
nktotal=list((kiwmesh[0]).values()).__len__()
niwtotal=list((kiwmesh[1]).values()).__len__()

In [4]:
eigvec = Gf(mesh=kiwmesh, target_shape=[4,4])
#initialize eigvec with random complex numbers of shape eigvec.data.shape
eigvec.data[:] = np.random.rand(*eigvec.data.shape) + 1j*np.random.rand(*eigvec.data.shape)
#normalize eigvec
eigvec /= np.sqrt(np.sum(eigvec.data * np.conj(eigvec.data)))

In [5]:
# we need a way to find the index of the mesh point corresponding to -k
def find_minusk_index(kmesh, k):
    for idx, km in enumerate(kmesh):
        if np.allclose((k + km.value) % (2*pi),[0,0,0]):
            return idx
    return None
find_minusk_index(kiwmesh[0],list(kiwmesh[0].values())[6])
# we need a way to find the index of the mesh point corresponding to k+kprime
def find_kpluskprime_index(kmesh, k, kprime):
    for idx, km in enumerate(kmesh):
        if np.allclose((k + kprime) % (2*pi),km.value):
            return idx
    return None
find_kpluskprime_index(kiwmesh[0],list(kiwmesh[0].values())[6],list(kiwmesh[0].values())[10])
# we need to store these indices in numpy arrays
minusk_index = np.zeros(len(kiwmesh[0]), dtype=int)
kpluskprime_index = np.zeros((len(kiwmesh[0]), len(kiwmesh[0])), dtype=int)
for idx, k in enumerate(kiwmesh[0]):
    minusk_index[idx] = find_minusk_index(kiwmesh[0], k.value)
    for idx2, kprime in enumerate(kiwmesh[0]):
        kpluskprime_index[idx, idx2] = find_kpluskprime_index(kiwmesh[0], k.value, kprime.value)

ok, now what about -iw and iw+iwp?

In [185]:
# the idea is that if the sum is outside the iOmegamesh, then we should ignore, otherwise find the Matsubara frequency index from the distance from the middle of the mesh, and add those two indices to get the appropriate index for the chi_expt_kiO mesh

for i in range(2400):
    for j in range(2400):
        mindexi= kiwmesh[1].first_index()+i
        mindexj= kiwmesh[1].first_index()+j
        if mindexi+mindexj<kiOmegamesh[1].last_index() and mindexi+mindexj>kiOmegamesh[1].first_index():
            print((list(kiwmesh[1].values())[i]+list(kiwmesh[1].values())[j],list(kiOmegamesh[1].values())[mindexi+mindexj+kiOmegamesh[1].first_index()]))

# no need to check further, this works

(-45405.411438979j, -45405.411438979j)
(-45367.51042776115j, -45367.51042776115j)
(-45329.60941654331j, -45329.60941654331j)
(-45291.70840532546j, -45291.708405325466j)
(-45253.807394107615j, -45253.807394107615j)
(-45215.90638288977j, -45215.90638288977j)
(-45178.00537167193j, -45178.00537167193j)
(-45140.10436045408j, -45140.10436045408j)
(-45102.20334923623j, -45102.20334923623j)
(-45064.30233801839j, -45064.30233801839j)
(-45026.401326800544j, -45026.401326800544j)
(-44988.5003155827j, -44988.5003155827j)
(-44950.59930436485j, -44950.59930436485j)
(-44912.698293147005j, -44912.698293147005j)
(-44874.79728192916j, -44874.79728192916j)
(-44836.89627071132j, -44836.89627071131j)
(-44798.99525949347j, -44798.995259493466j)
(-44761.09424827562j, -44761.09424827562j)
(-44723.19323705778j, -44723.19323705778j)
(-44685.292225839934j, -44685.292225839934j)
(-44647.39121462209j, -44647.39121462209j)
(-44609.49020340424j, -44609.49020340424j)
(-44571.589192186395j, -44571.589192186395j)
(-445

KeyboardInterrupt: 

In [156]:
(list(kiwmesh[1].values())[2005]+list(kiwmesh[1].values())[-2006])
# the index for -iw is just -idw-1
# what about iw+iwprime?
mindexi= kiwmesh[1].first_index()+i
mindexj= kiwmesh[1].first_index()+j
mindexi+mindexj+kiOmegamesh[1].first_index() #which maybe out of bounds unless mindexi+mindexj<kiOmegamesh[1].last_index() and mindexi+mindexj>kiOmegamesh[1].first_index()

32it [02:35,  4.86s/it]


KeyboardInterrupt: 

In [57]:
new_eigvec = Gf(mesh=kiwmesh, target_shape=[4,4])
# do while loop
old_eigvec = eigvec.copy()
bubble = np.zeros((4,4), dtype=complex)
for kpid in range(nktotal):
    for iwpd in tqdm(range(niwtotal)):
        bubble = np.einsum('af,be,ef->ab', G_kiw.data[minusk_index[kpid],-iwpd-1,:,:],G_kiw.data[kpid,iwpd,:,:],eigvec.data[kpid,iwpd,:,:])
        iwdarr=np.arange(-niO+1,niO)-iwpd+2*niw
        iwdarr = iwdarr[(iwdarr >= 0) & (iwdarr < niwtotal)]
        for kid in range(nktotal):
            for iwd in iwdarr: 
                # clear_output(wait=True)
                # display(f'Processing: kpid={kpid}, iwpd={iwpd}, kid={kid}, iwd={iwd}')
                new_eigvec.data[kid,iwd,:,:] += bubble * chi_expt_kiO.data[kpluskprime_index[kid,kpid],-2*niw+iwd+iwpd-niO] 
                # Gminuskp = G_kiw[:,:](-np.array(kp.value),-iwpd)
                # chi_expt_kiO.data[kpluskprime_index(),iw+iwp]
                # new_eigvec.data[kid,iwd,:,:] = np.einsum('af,be,ef->ab', G_kiw[:,:](-kp.value,-iwpd),G_kiw[:,:](kp,iwpd),eigvec[kp,iwp]) * chi_expt_kiO[k+kp,iw+iwp]

  0%|          | 0/256 [00:00<?, ?it/s]


100%|██████████| 35/35 [00:00<00:00, 69739.02it/s]


100%|██████████| 35/35 [00:00<00:00, 63221.64it/s]


100%|██████████| 35/35 [00:00<00:00, 116047.94it/s]


100%|██████████| 35/35 [00:00<00:00, 53694.46it/s]


100%|██████████| 35/35 [00:00<00:00, 88115.63it/s]


100%|██████████| 35/35 [00:00<00:00, 102586.05it/s]


100%|██████████| 35/35 [00:00<00:00, 77385.68it/s]


100%|██████████| 35/35 [00:00<00:00, 66007.48it/s]


100%|██████████| 35/35 [00:00<00:00, 59845.35it/s]


100%|██████████| 35/35 [00:00<00:00, 93563.19it/s]


100%|██████████| 35/35 [00:00<00:00, 60312.51it/s]


100%|██████████| 35/35 [00:00<00:00, 55167.47it/s]



100%|██████████| 35/35 [00:00<00:00, 109226.67it/s]


100%|██████████| 35/35 [00:00<00:00, 68026.25it/s]


100%|██████████| 35/35 [00:00<00:00, 57681.98it/s]


100%|██████████| 35/35 [00:00<00:00, 56745.51it/s]


100%|██████████| 35/35 [00:00<00:00, 78840.30it/s]


100%|██████████| 35/35 [00:00<00:00, 65653.24it/s]




KeyboardInterrupt: 

In [250]:
kiOmegamesh[1].last_index()

36

In [43]:
kmesh=kiwmesh[0]
np.array(list(kmesh.values()))[0]

array([0., 0., 0.])

In [54]:
gcopy=G_kiw.copy()
for k in kiwmesh[0]:
    for id, iw in enumerate(kiwmesh[1]):
        for kt in kiwmesh[0]:
            if np.allclose((k+kt),[0.,0.,0.]):
                print(G_kiw[1,1](tuple(-kt.value),-id))
                print(k+kt)

(0.000714154866660049-9.666854148713273e-06j)
[0. 0. 0.]
(0.000714154866660049+9.666854148713332e-06j)
[0. 0. 0.]
(0.0007131097798958798+2.8958123326489977e-05j)
[0. 0. 0.]
(0.0007110287558036478+4.812269438237915e-05j)
[0. 0. 0.]
(0.0007079298938762547+6.707814713411924e-05j)
[0. 0. 0.]
(0.0007038398529687655+8.57450641097898e-05j)
[0. 0. 0.]
(0.0006987932848945211+0.0001040481048138189j)
[0. 0. 0.]
(0.0006928321144694308+0.00012191696096996845j)
[0. 0. 0.]
(0.0006860046933411991+0.00013928716919297176j)
[0. 0. 0.]
(0.0006783648584899308+0.00015610076427781535j)
[0. 0. 0.]
(0.0006699709280878895+0.0001723067634482269j)
[0. 0. 0.]
(0.0006608846674796589+0.00018786147900624485j)
[0. 0. 0.]
(0.0006511702565352379+0.00020272866345037678j)
[0. 0. 0.]
(0.0006408932867830252+0.00021687949693549233j)
[0. 0. 0.]
(0.0006301198128597559+0.00023029243168554763j)
[0. 0. 0.]
(0.0006189154782650011+0.00024295291149854174j)
[0. 0. 0.]
(0.0006073447305229232+0.0002548529867621591j)
[0. 0. 0.]
(0.00059

KeyboardInterrupt: 

In [26]:
# G_kiw[:,:](tuple(-np.array([0.,0.39269908,0.])),-iwpd)
G_kiw[1,1]((0.,0.39269908,0.),-2)

(0.0007665155987496005+3.346656471520907e-05j)